In [33]:
import numpy as np
import pandas as pd
import datetime
import cx_Oracle
import os


def cp_query(pr_num, rev_num):
    
    selectstr1 = "(select distinct F.name as CustomerName, F.Customerid, B.name as LDC_Account, B.Accountid,"
    selectstr1 = "".join([selectstr1, "D.uidaccount, D.marketcode, A.Contractid, A.Revision "])
    selectstr1 = "".join([selectstr1, "from pwrline.account B, pwrline.lscmcontract A, pwrline.lscmcontractitem C, "])
    selectstr1 = "".join([selectstr1, "pwrline.acctservicehist D, pwrline.customer F "])
    selectstr1 = "".join([selectstr1, "where C.uidcontract=A.uidcontract and C.uidaccount=B.uidaccount and B.uidaccount=D.uidaccount "])
    selectstr1 = "".join([selectstr1, "and B.uidcustomer=F.uidcustomer and A.contractid='", pr_num, "' and A.revision=", rev_num, ") A "])

    s2str = "select distinct A.*, B.starttime, B.stoptime, B.overridecode as Tag_Type, B.val as Tag, B.strval as SOURCE_TYPE, B.lstime as Timestamp "
    s2str = "".join([s2str, "from pwrline.acctoverridehist B, ", selectstr1])
    s2str = "".join([s2str, "where A.uidaccount=B.uidaccount and (A.marketcode='PJM' OR  A.marketcode='NEPOOL' OR A.marketcode= 'NYISO' OR A.marketcode= 'MISO') "])
    s2str = "".join([s2str, "and (B.overridecode ='TRANSMISSION_TAG_OVRD' OR B.overridecode='CAPACITY_TAG_OVRD') "])
    s2str = "".join([s2str, "order by A.customername, B.overridecode, A.accountid, B.starttime"])

    uid = 'tesi_interface'
    pwd = 'peint88'

    ip = '172.25.152.125'
    port = '1700'
    service_name = 'tppe.mytna.com'
    dsn = cx_Oracle.makedsn(ip, port, service_name=service_name)

    return(s2str)


def OracleAPI(query):
    
    uid = 'tesi_interface'
    pwd = 'peint88'

    ip = '172.25.152.125'
    port = '1700'
    service_name = 'tppe.mytna.com'
    dsn = cx_Oracle.makedsn(ip, port, service_name=service_name)
    
    result_list = []
    con = cx_Oracle.connect(user = uid, password = pwd, dsn = dsn)
    cur = con.cursor()
    cur.execute(query)
    
    for result in cur:
        result_list.append(result)
        
    return(result_list)

#check tag coverage of PR start and end dates

def checkPRdates(data, PR_rev):
    
    pr_num = PR_rev.split('_')[0]
    rev_num = PR_rev.split('_')[1]

    query = "select starttime, stoptime from pwrline.lscmcontract where contractid='" + pr_num + "' and revision=" + rev_num

    PRstart, PRstop = OracleAPI(query)[0]


    missing_start = []
    missing_stop = []
    
    

    for acct in np.unique(data.LDC_Account):
    
        if min(data.StartTime[data.LDC_Account == acct]) > PRstart:
            print('PR start date before first Cap Tag, acct: ', acct)
            missing_start.append(acct)
    
        if max(data.StopTime[data.LDC_Account == acct]) < PRstop:
            print('PR end date after latest Cap Tag, acct: ', acct)
            missing_stop.append(acct)


    if len(missing_start) > 0:
        print("cap tags don't cover PR start for: ", missing_start)
    
    if len(missing_stop) > 0:
        print("cap tags don't cover PR end for: ", missing_stop)

    elif len(missing_start) == len(missing_stop) == 0:
        print("cap tags cover PR start & end dates for all accts in", PR_rev)
        
    return(np.union1d(missing_start, missing_stop))
    

def get_report(PR_rev):
    
    pr_num = PR_rev.split('_')[0]
    rev_num = PR_rev.split('_')[1]
    
    query = cp_query(pr_num, rev_num)
    output = OracleAPI(query)

    captag = pd.DataFrame.from_records(output)
    captag.columns = ['CustomerName', 'CustomerID', 'LDC_Account', 'AccountID', 'UIDaccount', 'MarketCode', 'ContractID', 'Revision', 'StartTime', 'StopTime', 'TagType', 'Tag', 'SourceType', 'TimeStamp']

    return(captag)
    
    
def export_report(PR_rev, Write_dir):  

    capreport = get_report(PR_rev)
    os.chdir(Write_dir)
    filename = 'CP_' + PR_rev + '.csv'
    dir_files = [file.split('.')[0] for file in os.listdir(Write_dir)]
    
    if filename.split('.')[0] not in dir_files:
        
        capreport.to_csv(filename, sep = ",", header = True, index = False)
        print('saved file as', filename)

    else:
        
        overwrite = input('file exists, overwrite? (Yes or No)')
        
        if overwrite.lower() == 'yes':
            capreport.to_csv(filename, sep = ",", header = True, index = False)
            print('saved file as', filename)
        
        elif overwrite.lower() == 'no':
            print('file ', filename, 'not saved.')
            
        else:
            print('command not recognized, input "yes" or "no".')
            

def batch_reports(PR_rev_list, Write_dir):
    
    for pr in PR_rev_list:
    
        print(' ')
        print('generating report...')
        report = get_report(pr)
    
        print(' ')
        print('------------------------')
        print(' ')
    
        print('checking dates, PR', pr)
        checkPRdates(report, pr)
    
        print(' ')
        print('------------------------')
        print(' ')
    
        print('exporting report...')
        export_report(pr, Write_dir)
    
        print(' ')
        print('------------------------')


In [26]:
report = pd.read_csv('ICAP_TAG_Power_4_Schools.csv', header = 0)
report.columns

Index(['CUSTOMERNAME', 'CUSTOMERID', 'LDC_ACCOUNT', 'ACCOUNTID', 'JURISCODE',
       'PROFILECODE', 'METERTYPE', 'LOSSCODE', 'ZONECODE', 'STARTTIME',
       'STOPTIME', 'USAGE', 'PEAK', 'CAPTAG_STARTTIME', 'CAPTAG_STOPTIME',
       'CAPTAG_DELTA', 'CUR_CAPTAG'],
      dtype='object')

In [29]:
report.STARTTIME = pd.to_datetime(report.STARTTIME)
report.STOPTIME = pd.to_datetime(report.STOPTIME)
report.head()

,CUSTOMERNAME,CUSTOMERID,LDC_ACCOUNT,ACCOUNTID,JURISCODE,PROFILECODE,METERTYPE,LOSSCODE,ZONECODE,STARTTIME,STOPTIME,USAGE,PEAK,CAPTAG_STARTTIME,CAPTAG_STOPTIME,CAPTAG_DELTA,CUR_CAPTAG
0,POWER 4 SCHOOLS ELECTRIC PROGRAM_AGG,1-IZS153,8037529480001137169,PJM_ATSIOE_08037529480001137169,OH,OE_C2,N,SECONDARY,FEOHIOOE,2019-06-01,2020-05-31,"199,272.01",87.52,6/1/18,5/31/2019,-29.9884,17.0961
1,POWER 4 SCHOOLS ELECTRIC PROGRAM_AGG,1-IZS153,8037529480001137168,PJM_ATSIOE_08037529480001137168,OH,OE_C2,N,SECONDARY,FEOHIOOE,2019-06-01,2020-05-31,"67,141.13",29.40,6/1/18,5/31/2019,-13.4137,9.7073
2,POWER 4 SCHOOLS ELECTRIC PROGRAM_AGG,1-IZS153,8037554831570000894,PJM_ATSICE_08037554831570000894,OH,CE_C2,N,SECONDARY,FEOHIOCE,2019-06-01,2020-05-31,"1,056,030.40",445.23,6/1/18,5/31/2019,-90.6808,273.6354
3,POWER 4 SCHOOLS ELECTRIC PROGRAM_AGG,1-IZS153,8006151431100086244,PJM_ATSICE_08006151431100086244,OH,CE_C2,N,SECONDARY,FEOHIOCE,2019-06-01,2020-05-31,"90,909.16",28.54,6/1/18,5/31/2019,0.9133,21.2582
4,POWER 4 SCHOOLS ELECTRIC PROGRAM_AGG,1-IZS153,8005799591590000899,PJM_ATSICE_08005799591590000899,OH,CE_C2,N,SECONDARY,FEOHIOCE,2019-06-01,2020-05-31,"72,842.82",22.38,6/1/18,5/31/2019,1.1047,19.8357


make sure StartTime and StopTime are python datetime values, ie 
    
    type(report.StartTime[0])

In [ ]:
def checkdatevalues(lildata, cap):
    
    if cap:
        
        starts = [!(dates.month == 6 and dates.day == 1) for dates in lildata.StartTime]
        stops = [!(dates.month == 5 and dates.day == 31) for dates in lildata.StopTime]
        
        probs = (starts or stops)
        
        return(lildata.iloc[probs,:])
    
    else:
        starts = [!(dates.month == 1 and dates.day == 1) for dates in lildata.StartTime]
        stops = [!(dates.month == 12 and dates.day == 31) for dates in lildata.StopTime]
        
        probs = (starts or stops)
        
        return(lildata.iloc[probs,:])

In [58]:
def datecheck(data):
    
    cap = data.loc[data.TagType.split('_')[0] == 'CAPACITY',:]
    trans = data.loc[data.TagType.split('_')[0] == 'TRANSMISSION',:]
    
    lastyr = datetime.datetime.strptime("2018", "%Y")
    thisyr = datetime.datetime.strptime("2019", "%Y")
    
    cur_cap = cap.loc[cap.StartTime.year >= lastyr, :]
    cur_trans = trans.loc[trans.StartTime.year >= thisyr,:]
    
    prob_caps = checkdatevalues(cur_cap, True)
    prob_trans = checkdatevalues(cur_trans, False)
    
    prob_caps.to_csv('Problem_Cap_Dates.csv', header = True, index = False)
    prob_trans.to_csv('Problem_Trans_Dates.csv', header = True, index = False)
    
    outside_accts = checkPRdates(data)
    outside = data.iloc[data.LDC_Account in outside_accts,:]
    outside.to_csv('Tag_Dates_Outside_PR.csv', header = True, index = False)
    
    bigcaptags = cur_cap.loc[cur_cap.Tag > 15,:]
    bigtranstag = cur_trans.loc[cur_trans.Tag > 15,:]
    
    
    
    
    
    


In [57]:
def capvalcheck(moredata):
    
    for acct in np.unique(moredata.LDC_ACCOUNT):
        acct_data = moredata.loc[moredata.LDC_ACCOUNT == acct,:]
        tags = moredata.Tag